In [ ]:
import os
import h5py
import stripy
import meshplex
import numpy as np
import pyvista as pv
from pathlib import Path
from scipy import interpolate
from PlateBoundaries import *
from TectonicEarth import Earth
from gospl.model import Model as sim
from gospl._fortran import definegtin
from EarthsAssistant import EarthAssist

In [ ]:
#Some code to check that I haven't broken Earth each whenever I edit it
if False:
    earth = Earth()
    earth.runTectonicSimulation()
    earth.animate(lookAtLonLat=[60, 20], cameraZoom=1.4)

# Gospl

Gospl is a python based numerical model for simulating various global scale erosion processes. It takes plate tectonic landscape displacements along with climatic data as input, and simulates *long-term catchment dynamic and drainage evolution as well as sedimentary basins formation*. Gospl simulations are run in two seperate stages, a backwards and a forward stage. The backwards stage is run first and is used to generate files that will help implement topological changes during the forward stage, and is otherwise optional. The forward stage is where the bulk of the erosion processes are simulated.


Gospl requires us to generate various files before the simulations can be run. The simulation properties are specified in two seperate YAML (Yet Another Markup Language) files, one for the forwards and one for the backwards simulations. Numerous NPZ (numpy zip) data files will be required for specifying elevation data, tectonic forces, or various other data for the simulation runs.

In this notebook, we will discuss the files required for running forward and backwards simulations, automatically generate those files from our Earth object, and analyize the output data produced by Gospl.



### Forward Simulations

All parameters and data files required by gospl are specified in the forward YML file. As such we will discuss each section of a YML file, and show how to generate the NPZ data files required for each section. For a more detailed documentation of *.yml* attributes used in *Gospl*, see the [official user guide](https://gospl.readthedocs.io/en/latest/user_guide/inputfile.html) documentation.

##### Domain

A *Gospl* input *yml* file begins by specifying the name, and the domain of the erosion simulation. *npdata* specifies the directory location of the initial surface mesh to be used by *Gospl*, and various other parameters. For forward simulations, the parameters *fast* and *forward* will be set to False. This section of the forward YML file takes the following form:

``` yml
name: Example input yml file

domain:
  npdata: 'ExampleInput/elev15Ma'
  flowdir: 5
  fast: False
  backward: False
  interp: 1
 ```
 
The main parameter of our interest here is *npdata*, which specifies the NPZ file location of the initial surface mesh to be used by Gospl. To illustrate the format that these elevation NPZ files need to take, we will inspect *ExampleInput/elev15Ma.npz*, which was taken from the Gospl *bfModel* notebook tutorial, and contains the following arrays:

- **v**: Vertices
- **c**: Cells
- **n**: Nearest Neighbours
- **z**: Elevations

In [ ]:
#Load npz file
npzFile = np.load('ExampleInput/elev15Ma.npz')
print(npzFile.files)

#Get arrays stored in file
vertices = npzFile['v']
cells = npzFile['c']
nearestNeighbours = npzFile['n']
elevations = npzFile['z']

#Print the shape of arrays within the npz file
print(vertices.shape)
print(cells.shape)
print(nearestNeighbours.shape)
print(elevations.shape)

We demonstrate how to read and visualize NPZ elevation files in the code bellow. As usual, we will exagerate the earth's topology to make topological features more visible. To create a pyvista mesh, we need to create a suitable list of faces.

In [ ]:
#Read npz file and return numpy arrays
def readNPZelevationFile(npzFileDir):
    npzFile = np.load(npzFileDir)
    elevation = npzFile['z']
    if len(elevation.shape) == 1:
        elevation = elevation[:, np.newaxis]
    return npzFile['v'], npzFile['c'], npzFile['n'], elevation

#Given the data from the NPZ file, we create an pyvista mesh of earth
def createMeshFromNPZdata(vertices, cells, heights, heightAmplification=30):
    faces = []
    exageratedRadius = vertices + heightAmplification * heights * vertices / np.max(vertices)
    for cell in cells:
        faces.append(3)
        faces.append(cell[0])
        faces.append(cell[1])
        faces.append(cell[2])
    earthMesh = pv.PolyData(exageratedRadius, faces)
    earthMesh['heights'] = heights
    return earthMesh

#Run the newly created functions
vertices, cells, neighbours, heights = readNPZelevationFile('ExampleInput/elev15Ma.npz')
earthMesh = createMeshFromNPZdata(vertices, cells, heights)
contour = earthMesh.contour([0])

#Display mesh
plotter = pv.PlotterITK()
plotter.add_mesh(earthMesh, scalars='heights')
plotter.add_mesh(contour, color="black", opacity=1.)
plotter.show()

We need to create a similar NPZ elevation file from our earth object. Because erosion algorithms used in Gospl depend on nearby neighbours of vertices, it benefits from using an icosahedral sphere (Icosphere), instead of the UV sphere that our earth has been based on so far. We begin by interpolating heights from our UV sphere to heights on an Icosphere, where the Icosphere is generated by the stripy library. The functions bellow takes an *Earth* object as input, and returns arrays of the vertices, cells and elevations for our *.npz* file.

In [ ]:
#Create an icosphere
def createIcosphere(subdivisions=6, radius=6378137):
    icosphere = stripy.spherical_meshes.icosahedral_mesh( 
                    refinement_levels = subdivisions,
                    include_face_points = False)
    icosphereXYZ = icosphere._points * radius
    icoCells = icosphere.simplices
    return icosphereXYZ, icoCells

#Create an Icosphere and interpolate our heights onto it
def earthToIcosphere(earth, subdivisions=6, iteration=-1):
    icosphereXYZ, icoCells = createIcosphere(subdivisions=subdivisions, radius=earth.earthRadius)
    
    #Interpolate heights
    earthHeights = earth.heightHistory[iteration]
    radLonLat = EarthAssist.cartesianToPolarCoords(icosphereXYZ)
    icoLonLat = np.stack((radLonLat[1], radLonLat[2]), axis=1)
    icoHeights = interpolate.griddata(earth.lonLat, earthHeights, icoLonLat, method='cubic')
    icoHeights = icoHeights[:, np.newaxis]
    return icosphereXYZ, icoCells, icoHeights

The last required array is that of the nearest neighbours IDs, which is done by the function bellow which was based on the *bfModel* notebook tutorial. We then define a function for writing the NPZ file. To confirm that everything works so far, we run the newly created functions and visualize the data from the newly generated NPZ file.

In [ ]:
#Create list of neighbour ids, based on bfModel notebook tutorial
def getNeighbourIds(icoXYZ, icoCells):
    Gmesh = meshplex.MeshTri(icoXYZ, icoCells)
    s = Gmesh.idx_hierarchy.shape
    a = np.sort(Gmesh.idx_hierarchy.reshape(s[0], -1).T)
    Gmesh.edges = {'points': np.unique(a, axis=0)}
    ngbNbs, ngbID = definegtin(len(icoXYZ), Gmesh.cells['points'], Gmesh.edges['points'])
    ngbIDs = ngbID[:,:8].astype(int)
    return ngbIDs

#Create an appropriate npz file from earth object and return the file name of the new npz file
def createNPZfromEarth(earth, outDirectory='GeneratedInputFiles/', iteration=-1, subdivisions=6):
    vertices, cells, heights = earthToIcosphere(earth, subdivisions=subdivisions)
    neighbours = getNeighbourIds(vertices, cells)
    
    #Create appropriate file name and save data as npz
    time = earth.timeHistory[iteration]
    fileName = '{}ElevationSubdivisions{}Time{}Mya'.format(outDirectory, subdivisions, time)
    np.savez_compressed(fileName, v=vertices, c=cells, n=neighbours.astype(int), z=heights)
    print(heights)
    return fileName + '.npz'
    

#Create instance of earth object to run newly created functions with
earth = Earth(
            startTime = 10,
            endTime = 0,
            deltaTime = 2,
            baseUplift = 2000,
            distTransRange = 1000000, 
            numToAverageOver = 10,
            earthRadius = 6378137.,
            useKilometres = False
)

#Create the npz file to be used by Gospl
elevationsFileName = createNPZfromEarth(earth)

#Read the newly created npz file as before to check that everything is working as expected
vertices, cells, neighbours, heights = readNPZelevationFile(elevationsFileName)
earthMesh = createMeshFromNPZdata(vertices, cells, heights)
contour = earthMesh.contour([0])

#Display the mesh
plotter = pv.PlotterITK()
plotter.add_mesh(earthMesh, scalars='heights')
plotter.add_mesh(contour, color="black", opacity=1.)
plotter.show()

##### Time

Now that we have discussed the domain of the gospl simulation and the required input files, the next set of parameters that need to be specified in the forward YML file is time, which has the following format:

``` yml
time:
  start: -10000000.
  end: 0.
  tout: 2000000.
  dt: 200000.
  tec: 2000000.
 ```
According to the [Gospl user documentation](https://gospl.readthedocs.io/en/latest/user_guide/inputfile.html), these parameters are described as:


- `start` is the model start time in years,

- `end` is the model end time in years,

- `tout` is the output interval used to create model outputs,

- `dt` is the model internal time step (the approach in *gospl* uses an implicit time step).

- `tec` is the tectonic timestep interval used to update the tectonic meshes and perform the required displacements.

##### Stream Power Law (SPL)

The Stream Power Law is a family of equations that governs erosion processes due to water flow (Eg. Rivers and rainfall). The defaults values are set as follows:

``` yml
spl:
  wfill: 100.
  sfill: 10.
  K: 3.e-8
  d: 0.42
```

##### Diffusion

Hillslope processes in gospl is defined using a classical diffusion law in which sediment deposition and erosion depend on slopes. The default parameters are as follows:

``` yml
diffusion:
  hillslopeKa: 0.02
  hillslopeKm: 0.2
  dstep: 5
  sedK: 100.
  sedKf: 200.
  sedKw: 300
```

##### Sea Level Forcing

Here we specify the sea level for the simulation. The *curve* parameter is optional and specifies the sea level over time, and if it is excluded from the *yml* file, then a constant default position of 0 will be used throughout the simulation.

``` yml
sea:
  position: 0.
  curve: 'data/sealevel.csv'
```

##### Tectonic

This section specifies a series tectonic forcing parameters in terms of displacements at each time step. They are specified as follows:

``` yml
tectonic:
  - start: -15000000.
    end: -14000000.
    mapH: 'input8/disp15Ma'
  - start: -14000000.
    end: -13000000.
    mapH: 'input8/disp14Ma'
  - start: -13000000.
    end: -12000000.
    mapH: 'input8/disp13Ma'
    ...
    ...
    ...
  - start: -1000000.
    end: 0.
    mapH: 'input8/disp1Ma'
```

Here the tectonic uplift force is specified by multiple *.npz* file at various times. These files contain *XYZ* coordinates representing the vertice's displacements at each specified timestep due to tectonic forces. These arrays need to be of shape (M, 3), where M is the number of vertices on our Icosphere. To visualize these files, we will use *pyvista* glyphs to create a vector field, where arrows have been placed just above earth to depict the direction and magnitude of the force. To avoid cluttering in our visualizations, we will place an arrow on a subset of the spheres vertices  by ignoring every N vertex.

To begin our analysis of tectonic displacement files, we will visualize an example displacement NPZ file provided by the Gospl bfModel notebook tutorial.

In [ ]:
#Read the npz tectonic file for displacements
def readNPZtectonicFile(npzFileDir):
    npzFile = np.load(npzFileDir)
    return npzFile['xyz']

#Creates a mesh object of the tectonic displacements vector field for visualizations
#We begin by normalizing the vectors so that their arrow sizes are consistent
def createTectonicVectorMesh(vertices, displacementXYZ, ignoreEveryNverts=50, vectorScale=600000):
    dispMagnitude = np.linalg.norm(displacementXYZ, axis=1)
    dispDirection = displacementXYZ / dispMagnitude[:, np.newaxis]
    dispMagnitude /= np.max(dispMagnitude)
    
    #Create an arrow template and place it at various locations just above earth's mesh
    arrow = pv.Arrow()
    reduceIndex = (np.arange(vertices.shape[0]) % ignoreEveryNverts == 0)
    vectorLocations = pv.PolyData(vertices[reduceIndex] * 1.02)
    vectorLocations['dispMag'] = dispMagnitude[reduceIndex]
    vectorLocations['dispDir'] = dispDirection[reduceIndex]
    forceVectors = vectorLocations.glyph(orient="dispDir", scale="dispMag", factor=vectorScale, geom=arrow)
    return forceVectors

#Provide an example for visualizing the tectonic displacements vector field
displacementXYZ = readNPZtectonicFile('ExampleInput/disp15Ma.npz')
vertices, cells, neighbours, heights = readNPZelevationFile('ExampleInput/elev15Ma.npz')
earthMesh = createMeshFromNPZdata(vertices, cells, heights)
forceVectors = createTectonicVectorMesh(vertices, displacementXYZ)

#Display the mesh
plotter = pv.PlotterITK()
plotter.add_mesh(earthMesh, scalars='heights')
plotter.add_mesh(forceVectors)
plotter.add_mesh(earthMesh.contour([0]), color="black", opacity=1.)
plotter.show()

We will now need to create a similar displacement NPZ file at each iteration of our Earth's simulation. Note that our Earth()'s vector field will look a bit different than that provided by the example tectonic file. This is fine, and is most likely due to the fact that we our velocities are based on different reference frames.

Note that in our *TectonicEarth.py* file for our Earth object, we have added a function *createTectonicDisplacements()* similar to the function bellow, which will store a history of tectonic diplacements as a list called *earth.tectonicDispHistory*. Later in this notebook we will read from *earth.tectonicDispHistory* instead of calling the function bellow.

In [ ]:
#Given an earth object, we create a tectonic displacement array of the latest iteration
#Note that this function needs to be called directly after we do an earth simulation step
def getTectonicDisplacements(earth):
    earthBeforeXYZ = earth.getEarthXYZ(amplifier=1, iteration=-2)
    
    #Get earth's XYZ after moving plates but before the remesh
    heightsAfter = earth.heights
    radius = heightsAfter + earth.earthRadius
    earthAfterXYZ = EarthAssist.polarToCartesian(radius, earth.movedLonLat[:, 0], earth.movedLonLat[:, 1])
    
    #Calculate the tectonic displacements, and set maxTectonicDisp
    tectonicDisp = (earthAfterXYZ - earthBeforeXYZ)  / earth.deltaTime
    return tectonicDisp

#Create instance of earth object to run newly created functions with
earth = Earth(
            startTime = 20,
            endTime = 0,
            deltaTime = 1,
            baseUplift = 2000,
            distTransRange = 1000000, 
            numToAverageOver = 10,
            earthRadius = 6378137.,
            useKilometres = False
)

#Dom some simulation steps
earth.doSimulationStep(earth.startTime)
#earth.doSimulationStep(earth.startTime - earth.deltaTime)
#earth.doSimulationStep(earth.startTime - 2 * earth.deltaTime)

#Create force vectors from tectonic displacements
tectonicDisp = getTectonicDisplacements(earth)
earthXYZ = earth.getEarthXYZ(amplifier=1, iteration=-1)
tectonicVectors = createTectonicVectorMesh(earthXYZ, tectonicDisp, ignoreEveryNverts=2, vectorScale=200000)

#Display the results
plotter = pv.PlotterITK()
earthMesh = earth.getEarthMesh(iteration=-1)
plotter.add_mesh(earthMesh, scalars=earth.heightHistory[-1])
plotter.add_mesh(tectonicVectors)
plotter.show()

To make this displacement field compatible with the Icosphere used by gospl, we need to interpolate this vector field onto the icosphere compatible with Gospl.

In [ ]:
#We interpolate the force field onto an Icosphere suitable for Gospl
def interpolateForces(earthLonLat, icosphereXYZ, forceXYZ):
    radLonLat = EarthAssist.cartesianToPolarCoords(icosphereXYZ)
    icoLonLat = np.stack((radLonLat[1], radLonLat[2]), axis=1)
    icoForce = interpolate.griddata(earthLonLat, forceXYZ, icoLonLat, method='cubic')
    return icoForce

Given an earth object where we have already run a tectonic simulation, we create a displacement NPZ file at each iteration of its simulation.

In [ ]:
#Create a function for writing tectonic displacement files given an earth object
def createTectonicDispNPZfiles(earth, icosphereXYZ, outputDirectory, subdivisions=6):
    fileNames = []
    for i, time in enumerate(earth.simulationTimes[:-1]):
        tectonicDisp = earth.tectonicDispHistory[i]
        icoForce = interpolateForces(earth.lonLat, icosphereXYZ, tectonicDisp)
        fileName = '{}ForceSubdivisions{}Time{}Mya'.format(outputDirectory, subdivisions, time)
        np.savez_compressed(fileName, xyz=icoForce)
        fileNames.append(fileName + '.npz')
    return fileNames

#Create instance of earth object to run newly created functions with
earth = Earth(
            startTime = 5,
            endTime = 0,
            deltaTime = 1,
            baseUplift = 2000,
            distTransRange = 1000000, 
            numToAverageOver = 10,
            earthRadius = 6378137.,
            useKilometres = False
)

#Create the elevation file to be used by Gospl
elevationsFileName = createNPZfromEarth(earth)
icosphereXYZ, icoCells, icoNeighbs, icoHeights = readNPZelevationFile(elevationsFileName)

#Run tectonic simulation
earth.runTectonicSimulation()
#earth.animate(lookAtLonLat=[60, 20])

#Create a directory for the generated files and then generate files
outputDir = './ExampleGeneratedNPZfiles'
if not os.path.isdir(outputDir):
    os.mkdir(outputDir)
fileNames = createTectonicDispNPZfiles(earth, icosphereXYZ, outputDir+'/')

In [ ]:
#Chose an iteration to visualize our created NPZ files with
iteration = 0
time = earth.simulationTimes[iteration]
fileName = outputDir + '/ForceSubdivisions6Time{}Mya.npz'.format(time)

#Create meshes for visualising the newly created files
displacementXYZ = readNPZtectonicFile(fileNames[iteration])
vertices, cells, neighbours, heights = readNPZelevationFile(elevationsFileName)
forceVectors = createTectonicVectorMesh(vertices, displacementXYZ, ignoreEveryNverts=10)
earthMesh = earth.getEarthMesh(iteration=iteration)

#Display the results
plotter = pv.PlotterITK()
plotter.add_mesh(earthMesh, scalars='heights')
plotter.add_mesh(forceVectors)
plotter.add_mesh(earthMesh.contour([0]), color="black", opacity=1.)
plotter.show()

##### Climate

The climate specifies the amount of rainfall at particular locations of the earth. Although Gospl allows us to specify it using another *.npz* file, we can also specify it to be uniformly distributed across the entire earth, which is done as follows.

``` yml
climate:
  - start: -15000000.
    uniform: 1.
```

##### Forcepaleo

For us to include topological changes due to tectonic forces into our simulation, we need to first run the backward Gospl models, and specify their output directories here.

``` yml
forcepaleo:
  dir: 'output-backward'
  steps: [10,5]
```

##### Output

Finally, we specify the output directory for the file generated by Gospl.

``` yml
output:
  dir: 'GosplOutput'
  makedir: False
```

### Backwards Simulations

For the most part, the files required in the backwards simulations are similar to those required by the forward simulations. They both require a similar YML file, and similar NPZ files. As such we will mainly discuss the differences that a backwards simulation requires to that of a forward simulation. Although the backwards simulations are optional for Gospl, they are required for the inclusion of topological changes in the final results, and we are therefore interested in them nonentheless. Without the backward simulations, gospl will only simulate errosion processes.

##### Domain

The most notable difference here is that the input npdata elevation file needs to be that of the end of the simulation, rather than from the begining. Additionally, the parameters *fast* and *backward* are now set to true.

``` yml
domain:
    npdata: 'input8/elev0Ma'
    flowdir: 5
    fast: True
    backward: True
    interp: 1
    overlap: 2
```

To briefly confirm that the input elevation file is of the format of what we expect, we provide a visualization of the example elevation file.

In [ ]:
vertices, cells, neighbours, heights = readNPZelevationFile('ExampleInput/elev0Ma.npz')
earthMesh = createMeshFromNPZdata(vertices, cells, heights)
contour = earthMesh.contour([0])

#Display mesh
plotter = pv.PlotterITK()
plotter.add_mesh(earthMesh, scalars='heights')
plotter.add_mesh(contour, color="black", opacity=1.)
plotter.show()

##### Tectonic

Looking at the backward YML file provided by the bfModel Gospl notebook tutorial, the tectonic section has the following structure:

``` yml
tectonic:
  - start: -15000000.
    end: -14000000.
    mapH: 'input8/backdisp1Ma'
  - start: -14000000.
    end: -13000000.
    mapH: 'input8/backdisp2Ma'
    ...
    
    ...
  - start: -1000000.
    end: 0.
    mapH: 'input8/backdisp15Ma'
```

Notice that the backdisp files go backwards in time, but for some reason the parameters *start* and *end* go forwards in time. I've also noticed that the forward displacement *npz* files and the backwards displacement files provided by the Gospl examples are almost identical, with the exception of multiplication of the displacements XYZ coordinates by negative 1. To see this uncomment the print statements in the code bellow.

In [ ]:
#Provide an example for visualizing the tectonic displacements vector field
displacementXYZ = readNPZtectonicFile('ExampleInput/disp15Ma.npz')
backDisplacementXYZ = readNPZtectonicFile('ExampleInput/backdisp15Ma.npz')
vertices, cells, neighbours, heights = readNPZelevationFile('ExampleInput/elev15Ma.npz')
earthMesh = createMeshFromNPZdata(vertices, cells, heights)
dispVectors = createTectonicVectorMesh(vertices, displacementXYZ)
backDispVectors = createTectonicVectorMesh(vertices, backDisplacementXYZ)

#print(displacementXYZ)
#print(backDisplacementXYZ)

#Display the mesh
plotter = pv.PlotterITK()
plotter.add_mesh(earthMesh, scalars='heights')
plotter.add_mesh(dispVectors)
#plotter.add_mesh(backDispVectors)
plotter.add_mesh(earthMesh.contour([0]), color="black", opacity=1.)
plotter.show()

# Automating the Gospl Process

Instead of manually creating the required files for Gospl, the GosplManager class defined bellow will take our Earth object in which the tectonic simulation has already been run as input and automatically generate all the required Gospl files in a new directory of the form "GosplRuns/runX" where X is some integer. It can then be used to run the Gospl simulation, and after discussing the methods, I will include methods for visualizing and animating the gospl results. Some of the functions in this class are based on our discussion above.

The class provides default values for each YML attribute, but they can all be changed by editing the *GosplManager* class attributes directly after instantiating a *GosplManager* object.

In [ ]:
class GosplManager:
    def __init__(self, earth,
                subdivisions=6,
                 mainOutputDirectory = 'GosplRuns'
                ):
        
        #Store attributes passed by class initiation and directory specifications
        self.earth = earth
        self.subdivisions = subdivisions
        self.mainOutputDirectory = mainOutputDirectory
        self.forwardOutputFolder = '/GosplOutputFiles'
        self.backwardOutputFolder = '/BackwardsOutput'
        
        #Icosphere for the creation of npz files
        self.icosphereXYZ, self.icoCells = self.createIcosphere(subdivisions=subdivisions, radius=earth.earthRadius)
        
        #============================== Default YML Attributes ======================================================
        #These attributes can be changed directly after initializing a GosplManager instance
        
        #Name attribute
        self.name = 'Automatically generated YML file'
        
        #Domain attributes
        #self.npdata = dataDirectory + '/Elevations{}Mya'.format(subdivisions, earth.startTime)
        self.npdataFormat = '{}/Elevations{}Mya'
        self.npdataBackFormat = '{}/BackwardsElevations{}Mya'
        self.flowdir = 5
        self.interp = 1
        
        #Time attributes
        self.gosplStepsAtEachIteration = 5
        self.start = earth.startTime * 1000000
        self.end = earth.endTime * 1000000
        self.tout = earth.deltaTime * 1000000
        self.dt = earth.deltaTime * 1000000 / self.gosplStepsAtEachIteration
        self.tec = earth.deltaTime * 1000000
        
        #Stream Power Law attributes (SPL)
        self.wfill = 100.
        self.sfill = 10.
        self.K = 3e-8
        self.d = 0.42
        
        #Diffusion attributes
        self.hillslopeKa = 0.02
        self.hillslopeKm = 0.2
        self.dstep = 5
        self.sedK = 100.
        self.sedKf = 200.
        self.sedKw = 300
        
        #Sea level attributes
        self.position = 0.
        
        #Climate attributes
        self.uniform = 1.
        
        #ForcePaleo attributes
        self.steps = [int(earth.startTime), int(earth.endTime)]#, int(earth.deltaTime)]
        
        #Output Attributes
        self.dirFormat = 'GosplOutput{}'
        self.makedir = False
        
    #============================== YML File Creation ======================================================
    #We generate a string containing all the content of the backwards and forwards YML file
    #For each section of the YML file, we have a seperate function creating a string for it,
    #We then combine all these sections into a single string and create the YML file
    def getNameString(self):
        return "\nname: {}\n\n".format(self.name)
    
    def getDomainString(self, backwards=False):
        domainFormat = "domain:\n  npdata: '{}'\n  flowdir: {}\n  fast: {}\n  backward: {}\n  interp: {}\n\n"
        npdataFormat = self.npdataFormat
        time = earth.startTime
        if backwards:
            npdataFormat = self.npdataBackFormat
            time = earth.endTime
        npdata = npdataFormat.format(self.npzFilesDirectory, time)
        domainString = domainFormat.format(
            npdata,
            self.flowdir,
            backwards,
            backwards,
            self.interp)
        return domainString
    
    def getTimeString(self):
        timeFormat = "time:\n  start: -{}\n  end: {}\n  tout: {}\n  dt: {}\n  tec: {}\n\n"
        self.dt = earth.deltaTime * 1000000 / self.gosplStepsAtEachIteration
        timeString = timeFormat.format(
            float(self.start),
            float(self.end),
            float(self.tout),
            float(self.dt),
            float(self.tec))
        return timeString
    
    def getSPLstring(self):
        splFormat = "spl:\n  wfill: {}\n  sfill: {}\n  K: {}\n  d: {}\n\n"
        splString = splFormat.format(
            self.wfill,
            self.sfill,
            self.getYMLscientificNotation(self.K),
            self.d)
        return splString
    
    def getDiffusionString(self):
        diffusionFormat = "diffusion:\n  hillslopeKa: {}\n  hillslopeKm: {}\n  dstep: {}\n  sedK: {}.\n  sedKf: {}.\n  sedKw: {}\n\n"
        diffusionString = diffusionFormat.format(
            self.hillslopeKa,
            self.hillslopeKm,
            self.dstep,
            int(self.sedK),
            int(self.sedKf),
            self.sedKw)
        return diffusionString
    
    def getSeaString(self):
        seaFormat = "sea:\n  position: {}\n\n"
        seaString = seaFormat.format(self.position)
        return seaString
    
    def getForwardTectonicString(self):
        tectonicString = "tectonic:\n"
        tectonicPartFormat = " - start: -{}.\n   end: -{}.\n   mapH: '{}/ForceSubdivisions{}Time{}Mya'\n"
        for time in self.earth.simulationTimes[:-1]:
            tectonicPart = tectonicPartFormat.format(
                1000000 * time, 
                1000000 * (time - earth.deltaTime),
                self.npzFilesDirectory,
                self.subdivisions,
                time)
            tectonicString += tectonicPart
        return tectonicString + '\n'
    
    def getBackwardsTectonicString(self):
        tectonicString = "tectonic:\n"
        tectonicPartFormat = " - start: -{}.\n   end: -{}.\n   mapH: '{}/BackwardsForceSubdivisions{}Time{}Mya'\n"
        times = self.earth.simulationTimes[:-1]
        for time in times:
            tectonicPart = tectonicPartFormat.format(
                1000000 * time, 
                1000000 * (time - earth.deltaTime),
                self.npzFilesDirectory,
                self.subdivisions,
                np.max(times) - time + earth.deltaTime)
            tectonicString += tectonicPart
        return tectonicString + '\n'
    
    def getClimateString(self):
        climateFormat = "climate:\n  - start: -{}.\n    uniform: {}\n\n"
        climateString = climateFormat.format(
            1000000 * self.earth.startTime,
            self.uniform)
        return climateString
    
    def getForcePaleoString(self):
        forcePaleoFormat = "forcepaleo:\n  dir: '{}'\n  steps: {}\n\n"
        forcePaleoString = forcePaleoFormat.format(
            self.thisRunDirectory + '/' + self.backwardOutputFolder,
            self.steps)
        return forcePaleoString
    
    def getOutputString(self, backwards=False):
        outputFormat = "output:\n  dir: '{}'\n  makedir: {}\n\n"
        outputFolder = self.forwardOutputFolder
        if backwards:
            outputFolder = self.backwardOutputFolder
        outputString = outputFormat.format(
            self.thisRunDirectory + outputFolder,
            self.makedir)
        return outputString
    
    #Create a string containing all the content of the YML file
    def getForwardYMLstring(self):
        name = self.getNameString()
        domain = self.getDomainString()
        time = self.getTimeString()
        spl = self.getSPLstring()
        diffusion = self.getDiffusionString()
        sea = self.getSeaString()
        tectonic = self.getForwardTectonicString()
        climate = self.getClimateString()
        forcePaleo = self.getForcePaleoString()
        output = self.getOutputString()
        return name + domain + time + spl + diffusion + sea + tectonic + climate + forcePaleo + output
    
    #Create a string containing all the content of the YML file
    def getBackwardYMLstring(self):
        name = self.getNameString()
        domain = self.getDomainString(backwards=True)
        time = self.getTimeString()
        spl = self.getSPLstring()
        diffusion = self.getDiffusionString()
        sea = self.getSeaString()
        tectonic = self.getBackwardsTectonicString()
        climate = self.getClimateString()
        output = self.getOutputString(backwards=True)
        return name + domain + time + spl + diffusion + sea + tectonic + climate + output
    
    #If x is given in scientific notation, return x as string scientific notation compatible with YML files.
    @staticmethod
    def getYMLscientificNotation(x):
        if 'e' in str(x):
            x = str(x).split('e-')
            x = '{}.e-{}'.format(x[0], x[1])
        return x
    #============================== Create Directories and YML Files ==============================================
    def makeDirectories(self):
        
        #Create the main output directory if it does not already exist
        if not os.path.isdir(self.mainOutputDirectory):
            os.mkdir('./{}'.format(self.mainOutputDirectory))
        
        #Create subdirectory for this particular Gospl run
        runNumber = 1
        thisRunDirectory = './{}/run{}'.format(self.mainOutputDirectory, runNumber)
        while os.path.isdir(thisRunDirectory):
            runNumber += 1
            thisRunDirectory = './{}/run{}'.format(self.mainOutputDirectory, runNumber)
        self.thisRunDirectory = thisRunDirectory
        os.mkdir(thisRunDirectory)
        
        #Create directory for npz files
        self.npzFilesDirectory = thisRunDirectory + '/NPZfiles'
        os.mkdir(self.npzFilesDirectory)
        
        #Create the forward YML file
        ymlForwardContent = self.getForwardYMLstring()
        self.ymlForwardDirectory = thisRunDirectory + '/forward.yml'
        ymlForward = open(self.ymlForwardDirectory, 'w')
        ymlForward.write(ymlForwardContent)
        ymlForward.close()
        
        #Create the backwards YML file
        ymlBackwardContent = self.getBackwardYMLstring()
        self.ymlBackwardDirectory = thisRunDirectory + '/Backward.yml'
        ymlBackward = open(self.ymlBackwardDirectory, 'w')
        ymlBackward.write(ymlBackwardContent)
        ymlBackward.close()
    
    #============================== Create Domain Elevations NPdata ======================================================
    #Create an icosphere
    @staticmethod
    def createIcosphere(subdivisions=6, radius=6378137):
        icosphere = stripy.spherical_meshes.icosahedral_mesh( 
                        refinement_levels = subdivisions,
                        include_face_points = False)
        icosphereXYZ = icosphere._points * radius
        icoCells = icosphere.simplices
        return icosphereXYZ, icoCells
    
    #Create an icosphere and interpolate earth heights onto it
    def getIcoHeights(self, iteration=-1):
        earthHeights = self.earth.heightHistory[iteration]
        radLonLat = EarthAssist.cartesianToPolarCoords(self.icosphereXYZ)
        icoLonLat = np.stack((radLonLat[1], radLonLat[2]), axis=1)
        icoHeights = interpolate.griddata(self.earth.lonLat, earthHeights, icoLonLat, method='cubic')
        icoHeights = icoHeights[:, np.newaxis]
        return icoHeights
    
    #Create list of neighbour ids based on bfModel notebook tutorial
    def getNeighbourIds(self):
        Gmesh = meshplex.MeshTri(self.icosphereXYZ, self.icoCells)
        s = Gmesh.idx_hierarchy.shape
        a = np.sort(Gmesh.idx_hierarchy.reshape(s[0], -1).T)
        Gmesh.edges = {'points': np.unique(a, axis=0)}
        ngbNbs, ngbID = definegtin(len(self.icosphereXYZ), Gmesh.cells['points'], Gmesh.edges['points'])
        ngbIDs = ngbID[:,:8].astype(int)
        return ngbIDs
    
    #Create the domains npdata elevations file
    def createDomainNPdataFile(self):
        
        #Create data file for forward model
        heights = self.getIcoHeights(iteration=0)
        neighbours = self.getNeighbourIds()
        fileName = self.npdataFormat.format(self.npzFilesDirectory, earth.startTime)
        np.savez_compressed(fileName, v=self.icosphereXYZ, c=self.icoCells, n=neighbours.astype(int), z=heights)
        
        #Create data file for backward model
        heights = self.getIcoHeights(iteration=-1)
        fileName = self.npdataBackFormat.format(self.npzFilesDirectory, earth.endTime)
        np.savez_compressed(fileName, v=self.icosphereXYZ, c=self.icoCells, n=neighbours.astype(int), z=heights)
    
    #============================== Create Tectonic Displacements Files =============================================
    #We interpolate the force field onto an Icosphere suitable for Gospl
    def interpolateForces(earthLonLat, icosphereXYZ, forceXYZ):
        radLonLat = EarthAssist.cartesianToPolarCoords(icosphereXYZ)
        icoLonLat = np.stack((radLonLat[1], radLonLat[2]), axis=1)
        icoForce = interpolate.griddata(earthLonLat, forceXYZ, icoLonLat, method='cubic')
        return icoForce
    
    #Create tectonic force displacement files
    def createTectonicDispNPZfiles(self):
        times = self.earth.simulationTimes[:-1]
        for i, time in enumerate(times):
            tectonicDisp = self.earth.tectonicDispHistory[i]
            icoForce = interpolateForces(self.earth.lonLat, self.icosphereXYZ, tectonicDisp)
            
            #Forward displacement files
            fileName = '{}/ForceSubdivisions{}Time{}Mya'.format(self.npzFilesDirectory, self.subdivisions, time)
            np.savez_compressed(fileName, xyz=icoForce)
            
            #Backward displacement files
            fileName = '{}/BackwardsForceSubdivisions{}Time{}Mya'.format(self.npzFilesDirectory, self.subdivisions, time)
            np.savez_compressed(fileName, xyz=-icoForce)
    
    #Run the gospl simulation
    def runGosplSimulation(self):
        
        #Run backward model
        mod = sim(self.ymlBackwardDirectory, False, False)
        mod.runProcesses()
        mod.destroy()
        
        #Run forward model
        mod = sim(self.ymlForwardDirectory, False, False)
        mod.runProcesses()
        mod.destroy()
    
    #After creating a GosplManager object, this function will do everything to run a simulation
    #Alternatively you can call these functions individually
    def createAllFilesAndRunSimulation(self):
        self.makeDirectories()
        self.createDomainNPdataFile()
        self.createTectonicDispNPZfiles()
        self.runGosplSimulation()

Now that we have created the *GosplManager* class, we demonstrate bellow how to use it. By default, the results should be found in a subdirectory of the *GosplRuns* directory found in the same folder of this notebook. These directories will also be automatically generated. 

Note that for some reason, Gospl only works with simulations with deltaTime set to 1 million years. Also, for some reason, gospl seems to simulate each timestep twice, which seems to be a bug.

In [ ]:
#Create instance of earth object to the gospl manager with
earth = Earth(
            startTime = 5,
            endTime = 0,
            deltaTime = 1,
            baseUplift = 2000,
            distTransRange = 1000000, 
            numToAverageOver = 10,
            earthRadius = 6378137.,
            useKilometres = False
)
#Run Earth's tectonic simulation before we run Gospl
earth.runTectonicSimulation()

#Create gospl object and run it
gosplMan = GosplManager(earth)

#Change gospl parameters like this
gosplMan.gosplStepsAtEachIteration = 2

#Run the gospl simulation
gosplMan.createAllFilesAndRunSimulation()

#Alternatively, we can run particular parts of the simulation like this (for debuggin)
if False:
    gosplMan = GosplManager(earth)
    gosplMan.makeDirectories()
    gosplMan.createDomainNPdataFile()
    gosplMan.createTectonicDispNPZfiles()
    gosplMan.runGosplSimulation()

# Analyzing the Gospl output files

After running a Gospl simulation, a folder named 'h5' should be generated within the output directory specified by the *.yml* file. This folder contains the output data generated by Gospl. It contains a one *topology.p0.h5* file with mesh data of the earth, and a series of files with the naming scheme *gospl.{}.p0.h5* containing simulation output data at each time step *tout*, where *tout* was specified by the *.yml* file, and the symbol *{}* in the naming scheme represents the simulation iteration.

Each file of the format *gospl.{}.p0.h5* contains the following data for each vertex on the earth's sphere:

- `elev`: The height of the vertex

- `erodep`: The amount of soil deposited at this vertex

- `fillAcc`: The amount of water on a vertex

- `flowAcc`: The flow of water on a vertex

- `rain`: The amount of rainfall on a given vertex

- `sedLoad`: The amount of sediment being carried by the water 

Note that the above attributes are not documented in the Gospl documentations and are based on my best guess. (Note to Tristan: You might want to add these to your user documentations).

We now provide an example of how to visualize the gospl output data files. Note that for the purpose of visualizations, we have raised some attributes by the power of 0.25 to make features more visible. To specify the attribute to visualize, use the dropdown menu located in the top left of the render window, alternatively we can change the 'scalars' argument in the *plotter.add_mesh()* function call.

In [ ]:
#The Gospl file contains simulation output data at particular iterations during the simulation
def readGosplFile(fileDir):
    gosplDict = {}
    with h5py.File(fileDir, "r") as f:
        for key in f.keys():
            gosplDict[key] = np.array(f[key])
    return gosplDict

#Using the gospl data, we create a pyvista mesh
def createGosplDataMesh(gosplFilenamePattern, iteration, radius=6378137, heightAmplification=30, subdivisions=6):
    gosplData = readGosplFile(gosplFilenamePattern.format(iteration))
    icoXYZ, icoCells = createIcosphere(subdivisions=subdivisions, radius=earth.earthRadius)
    outputMesh = createMeshFromNPZdata(icoXYZ, icoCells, gosplData['elev'], heightAmplification=heightAmplification)
    
    #Store gospl data as mesh attributes
    outputMesh['elev'] = gosplData['elev'] 
    outputMesh['erodep'] = gosplData['erodep'] 
    outputMesh['fillAcc'] = gosplData['fillAcc']**0.25
    outputMesh['flowAcc'] = gosplData['flowAcc']**0.25
    outputMesh['rain'] = gosplData['rain'] 
    outputMesh['sedLoad'] = gosplData['sedLoad']**0.25
    return outputMesh

#Specify an iteration of the Gospl simulation that we wish to visualize
iteration = 5
outputDir = gosplMan.thisRunDirectory + '/GosplOutputFiles/h5/'
gosplFilenamePattern = outputDir + 'gospl.{}.p0.h5'
outputMesh = createGosplDataMesh(gosplFilenamePattern, iteration)

#Plot the results
plotter = pv.PlotterITK()
plotter.add_mesh(outputMesh, scalars='flowAcc')
plotter.show()

The function bellow can be used to create an animation of the gospl results

In [ ]:
#Animate the results produced by Gospl
def animateGosplOutput(outputDir, subdivisions,
                       scalarAttribute = 'elev',
                       gosplFilenamePattern = 'gospl.{}.p0.h5',
                       movieOutputFileName='GosplAnimation.mp4',
                       lookAtLonLat = np.array([60, 20]),
                       cameraZoom = 1.4,
                       framesPerIteration = 8):
    
    #Set up directories, get number of file for animation, and initialise the animation mesh
    outputPath = Path(outputDir)
    numOfFiles = len(list(outputPath.glob('gospl.*.p0.h5')))
    fileNamePattern = outputDir + gosplFilenamePattern
    earthMesh = createGosplDataMesh(fileNamePattern, 0, subdivisions=subdivisions)
    
    #Set up plotter object and camera position for animation
    plotter = pv.Plotter()
    plotter.add_mesh(earthMesh, scalars=scalarAttribute, cmap='gist_earth')
    plotter.camera_position = 'yz'
    plotter.camera.zoom(cameraZoom)
    plotter.camera.azimuth = 180 + lookAtLonLat[0]
    plotter.camera.elevation = lookAtLonLat[1]
    plotter.show(auto_close=False, window_size=[800, 608])
    plotter.open_movie(movieOutputFileName)
    for i in range(framesPerIteration):
        plotter.write_frame()
    
    #Iterate through files and write animation frames
    for i in range(numOfFiles-1):
        newMesh = createGosplDataMesh(fileNamePattern, i+1, subdivisions=subdivisions)
        plotter.update_coordinates(newMesh.points, mesh=earthMesh)
        plotter.update_scalars(newMesh[scalarAttribute], render=False, mesh=earthMesh)
        for i in range(framesPerIteration):
            plotter.write_frame()
    plotter.close()
    return

#Create a gospl animation
animateGosplOutput(outputDir, 6, scalarAttribute='flowAcc')

# Note To Self

As of now, when I run the tectonic earth simulations for too long, the resulting mountains get too big. Although I have made attempts at solving this issue in the past, I feel like erosion processes is what I should be using. I should modify this notebook and run a new gospl simulation after each tectonic earth simulation, and see if the Gospl errosion algorithm prevents my mountains from growing too high.